# BB84 — Bob Role

This notebook creates the `bob_notebook.py` server script and shows how to run it.

**Pre-requisites**:

- Python 3.9+
- Install: `pip install qiskit qiskit-ibm-runtime flask requests numpy cryptography`

Make sure QiskitRuntimeService credentials are saved on machines that will call IBM backends.

In [ ]:
# Write the role script to disk
script = r"""
# bob.py
from flask import Flask, request, jsonify
import numpy as np, time, requests
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

app = Flask(__name__)

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibm_fez")
sampler = Sampler(backend=backend)

N = 0
bob_bases = []
received_preps = []
bob_measurements = {}

@app.route("/receive_from_eve", methods=["POST"])
def receive_from_eve():
    global received_preps, bob_bases, N, bob_measurements
    data = request.json
    received_preps = data.get("preps", [])
    N = len(received_preps)
    np.random.seed(int(time.time()) % 100000)
    bob_bases = np.random.randint(2, size=N).tolist()
    circuits = []
    for i, item in enumerate(received_preps):
        bit = item["bit"]
        basis = item["basis"]
        qc = QuantumCircuit(1,1)
        if bit == 1:
            qc.x(0)
        if basis == 1:
            qc.h(0)
        if bob_bases[i] == 1:
            qc.h(0)
        qc.measure(0,0)
        circuits.append(qc)
    job = sampler.run(circuits)
    result = job.result()
    outcomes = []
    try:
        for k in range(len(circuits)):
            counts = result.results[k].data.counts
            bit = int(list(counts.keys())[0])
            outcomes.append(bit)
    except Exception:
        try:
            for k in range(len(circuits)):
                counts = result.get_counts(k)
                bit = int(list(counts.keys())[0])
                outcomes.append(bit)
        except Exception as e:
            print("Parsing error, fallback random:", e)
            outcomes = list(np.random.randint(2, size=N))
    for i, item in enumerate(received_preps):
        idx = item["idx"]
        bob_measurements[idx] = int(outcomes[i])
    return jsonify({"status":"measured", "n": N})

@app.route("/receive_alice_bases", methods=["POST"])
def receive_alice_bases():
    alice_bases = request.json.get("bases", [])
    if len(alice_bases) == 0:
        return jsonify({"error":"no bases provided"}), 400
    indices = [item["idx"] for item in received_preps]
    a_bases_aligned = [int(alice_bases[i]) for i in range(len(indices))]
    b_bases_aligned = [int(bob_bases[i]) for i in range(len(indices))]
    alice_bits_aligned = [int(item.get("bit")) for item in received_preps]
    bob_bits_aligned = [bob_measurements[item["idx"]] for item in received_preps]
    sift_mask = [1 if a_bases_aligned[i] == b_bases_aligned[i] else 0 for i in range(len(indices))]
    sifted_alice = [alice_bits_aligned[i] for i in range(len(indices)) if sift_mask[i]]
    sifted_bob = [bob_bits_aligned[i] for i in range(len(indices)) if sift_mask[i]]
    if len(sifted_alice) == 0:
        qber = None
    else:
        errors = sum([1 for i in range(len(sifted_alice)) if sifted_alice[i] != sifted_bob[i]])
        qber = errors / len(sifted_alice)
    return jsonify({"sifted_len": len(sifted_alice), "qber": qber,
                    "sifted_alice": sifted_alice, "sifted_bob": sifted_bob})

@app.route("/receive_ciphertext", methods=["POST"])
def receive_ciphertext():
    data = request.json
    ciphertext = bytes(data.get("ciphertext", []))
    return jsonify({"ciphertext_len": len(ciphertext), "note": "decrypt on client after retrieving key via sifting API"})

if __name__ == "__main__":
    print("Bob server running on port 5002")
    app.run(host="0.0.0.0", port=5002)

"""
path = '/mnt/data/bob_notebook.py'
with open(path, 'w', encoding='utf-8') as f:
    f.write(script)
print('Wrote', path)

print('\nEdit the host addresses inside the script file, then run in a terminal:')
print('python', path)


Bob receives requests. Ensure QiskitRuntimeService credentials are saved here.